https://ba.hitomi.la/webp/1686733202/414/18df50f9540ce9c3200c84860f29bdb1bbd4c20cb38becab12624196b5e819e1.webp 怎么构造的？ 
1. 1686733202可能是时间戳，是目前整点的吧,我测试发现17:44时对应时间戳是17:00的
2. 414那部分是将倒数第一位和倒数二三换位后转十六进制int('19e',16)
2. 最后一部分是哈希值，可以通过解析https://ltn.hitomi.la/galleries/2113970.js 获得 其中最后的数字是画廊id
1. aa还是ba有自己的判断逻辑 具体见https://ltn.hitomi.la/gg.js

## 思路
注意以下必须要注意referer 不然会报错！
1. 用户输入画廊链接
2. 提取链接的galleryid
3. 请求js地址，返回数据https://ltn.hitomi.la/galleries/画廊的id.js
3. 请求gg.js,https://ltn.hitomi.la/gg.js 返回映射表，时间戳
4. 解析返回数据，格式化，返回info并存储本地cache
    > 需要注意这步关键是获取哈希值
    
6. 根据哈希值获取subdomain即十六进制数部分
7. 根据返回映射表判断链接
8. 以上 获取了所有图片的下载链接
9. 下载为jpg格式 可以多线程，只需要传入url

这段是构造是aa还是ba

In [40]:
import re
import requests
import json
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
from requests.exceptions import HTTPError

headers={
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Mobile Safari/537.36',
    'referer':'https://hitomi.la/',
}
class Util:
    def get_gallery_id(url:str):
        '''
        根据主页获取galleryid
        xxx-388036-211xxxx.html >> 211xxxx
        '''
        pattern=r'.*?-(?P<gallery_id>\d+).html.*'
        match=re.match(pattern=pattern,string=url).group('gallery_id')
        return match

    def get_image_infos(gallery_id:str):
        '''
        根据画廊id,请求js网址获得即该作品的所有信息,
        其中files被更改为hash存储所有哈希值
        获取hash_list: images_data['hash']
        '''
        js_url=f'https://ltn.hitomi.la/galleries/{gallery_id}.js' 
        #注意 访问改连接很大概率会被禁ip,谨慎行事
        with requests.get(js_url,headers=headers) as response:
            data=response.text
            match=re.search(pattern=r'.*?({.*})',string=data).group(1) #去除不需要的部分
            images_data=json.loads(match) #转为python格式,是字典
            images_data['hash']=[hash['hash'] for hash in images_data['files']] #获取哈希值列表
            del images_data['files']
        return images_data

    def get_time_info():
        '''获取gg.js即时间戳信息和映射表信息'''
        infos={}
        js_url='https://ltn.hitomi.la/gg.js'
        with requests.get(js_url,headers=headers) as response:
            if response.status_code!=200:
                logging.error(f'获取{js_url}信息时出错')
                raise HTTPError
            time_data=response.text
            timestamp=re.search(pattern=r'b:\s+\'(\d+)/\'',string=time_data).group(1)
            mapping=re.findall(pattern=r'case (\d+?):',string=time_data)
            infos={
                'mapping':mapping,
                'timestamp':timestamp,
            }
            return infos
    def get_subdomains(hash_list:list):
        '''
        根据哈希列表，返回一个对应的subdomain列表,即对应的十六进制数
        规则：xxx9e1 >>> 9e 1 换位>>> 19e
        '''
        subdomains=[str(int(hash[-1]+hash[-3:-1],16)) for hash in hash_list]
        return subdomains
    
    def get_mapping_result(subdomains,mapping):
        '''根据subdomain列表和映射表mapping
        获取各个对应的是aa还是bb
        subdomains=['123','456','1892']
        mapping=['2544','3652','2014']
        get_mapping_result(subdomains,mapping)
        '''
        return ['b' if subdomain in mapping else 'a' for subdomain in subdomains ]


images_data=Util.get_image_infos('2113970')
time_info=Util.get_time_info()
subdomains=Util.get_subdomains(images_data['hash'])
result=Util.get_mapping_result(subdomains,time_info['mapping'])


In [42]:
subdomains

['709',
 '414',
 '984',
 '849',
 '3953',
 '2959',
 '309',
 '1511',
 '2994',
 '1685',
 '267',
 '157',
 '2876',
 '2860',
 '163',
 '734',
 '2516',
 '2378',
 '2660',
 '1784',
 '3785',
 '3824',
 '1064',
 '3981',
 '334',
 '913',
 '1370',
 '1372',
 '609',
 '1256',
 '3258',
 '2566',
 '3096',
 '1502',
 '469',
 '2722',
 '361',
 '1182',
 '954',
 '749',
 '1257',
 '515',
 '3068',
 '1279']

In [41]:
result

['a',
 'a',
 'b',
 'a',
 'a',
 'a',
 'b',
 'a',
 'b',
 'a',
 'a',
 'b',
 'b',
 'b',
 'a',
 'a',
 'b',
 'a',
 'b',
 'b',
 'b',
 'b',
 'a',
 'a',
 'a',
 'b',
 'b',
 'b',
 'a',
 'b',
 'a',
 'b',
 'a',
 'a',
 'b',
 'b',
 'b',
 'a',
 'a',
 'a',
 'a',
 'a',
 'b',
 'a']

In [37]:
time_info['mapping']

['4006',
 '2737',
 '1826',
 '3795',
 '685',
 '1382',
 '3806',
 '3016',
 '3534',
 '1529',
 '2974',
 '2903',
 '3961',
 '1988',
 '3800',
 '3493',
 '2852',
 '2625',
 '242',
 '455',
 '3408',
 '404',
 '3812',
 '3453',
 '446',
 '4083',
 '2300',
 '2190',
 '3661',
 '3102',
 '2202',
 '3036',
 '634',
 '2203',
 '2609',
 '3168',
 '2401',
 '569',
 '83',
 '3631',
 '133',
 '1623',
 '2155',
 '1554',
 '1420',
 '2351',
 '2830',
 '2254',
 '880',
 '2666',
 '1711',
 '2206',
 '728',
 '3000',
 '1510',
 '3530',
 '1817',
 '428',
 '3891',
 '2540',
 '4059',
 '2149',
 '2746',
 '109',
 '3341',
 '4015',
 '1652',
 '115',
 '4007',
 '1938',
 '1772',
 '679',
 '1254',
 '3935',
 '3823',
 '2896',
 '1781',
 '1001',
 '1978',
 '958',
 '2305',
 '3813',
 '3132',
 '1128',
 '32',
 '1178',
 '3344',
 '35',
 '1690',
 '1568',
 '2884',
 '3076',
 '525',
 '3642',
 '2951',
 '294',
 '177',
 '3085',
 '2565',
 '1200',
 '396',
 '2810',
 '2262',
 '1990',
 '403',
 '3773',
 '467',
 '3996',
 '2258',
 '941',
 '1447',
 '1715',
 '84',
 '674',
 '195